In [1]:
import os
import openai
import json
import math

# Load environment variables
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

def calculate_cylinder_volume(radius, height):
    """
    Calculates the volume of a cylinder.

    Args:
        radius (float): The radius of the cylinder's base.
        height (float): The height of the cylinder.

    Returns:
        str: A JSON string containing the radius, height, and the calculated volume.
    """
    try:
        # Volume of a cylinder formula: V = pi * r^2 * h
        volume = math.pi * (radius ** 2) * height
        result = {
            "radius": radius,
            "height": height,
            "volume": volume,
        }
    except Exception as e:
        result = {
            "radius": radius,
            "height": height,
            "error": str(e),
        }
    return json.dumps(result)

# Define the function for the LLM to call
functions = [
    {
        "name": "calculate_cylinder_volume",
        "description": "Calculates the volume of a cylinder given its radius and height.",
        "parameters": {
            "type": "object",
            "properties": {
                "radius": {
                    "type": "number",
                    "description": "The radius of the cylinder's base.",
                },
                "height": {
                    "type": "number",
                    "description": "The height of the cylinder.",
                },
            },
            "required": ["radius", "height"],
        },
    }
]

# Create a user message to trigger the function
messages = [
    {
        "role": "user",
        "content": "What is the volume of a cylinder with a radius of 5 and a height of 10?"
    }
]

# Call the ChatCompletion endpoint
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

# Extract and process the response
response_message = response["choices"][0]["message"]
print("LLM Response Message:")
print(response_message)
print("-" * 20)

# Check if the LLM chose to call the function
if response_message.get("function_call"):
    function_name = response_message["function_call"]["name"]
    function_args = json.loads(response_message["function_call"]["arguments"])
    
    # Verify the function name and call it
    if function_name == "calculate_cylinder_volume":
        print("Calling the 'calculate_cylinder_volume' function...")
        volume_result = calculate_cylinder_volume(
            radius=function_args.get("radius"), 
            height=function_args.get("height")
        )
        print("Function Output:")
        print(volume_result)
        
        # Append the function's output to the messages list
        messages.append(response_message)
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": volume_result,
            }
        )
        
        # Call the LLM again with the function's output to get a final response
        print("-" * 20)
        print("Calling LLM with function output for a final response...")
        final_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        
        print("Final LLM Response:")
        print(final_response["choices"][0]["message"]["content"])
    else:
        print(f"The LLM called an unexpected function: {function_name}")
else:
    print("The LLM did not call a function.")
    print("Final LLM Response:")
    print(response_message["content"])

LLM Response Message:
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "calculate_cylinder_volume",
    "arguments": "{\"radius\":5,\"height\":10}"
  },
  "refusal": null,
  "annotations": []
}
--------------------
Calling the 'calculate_cylinder_volume' function...
Function Output:
{"radius": 5, "height": 10, "volume": 785.3981633974483}
--------------------
Calling LLM with function output for a final response...
Final LLM Response:
The volume of a cylinder with a radius of 5 and a height of 10 is approximately 785.40 cubic units.
